In [29]:
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy.exc
from urllib import parse
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


# Define Database Connection

CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server&AUTOCOMMIT=TRUE'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
conn = engine.connect()


In [47]:

def add_obs(df):
    """Add new Obs Spare Parts to the Obs Spare Parts Table"""
    table_name = 'tblObsSpares'
    df.to_sql(
        name=table_name,
        con=engine,
        schema='eng',
        if_exists='append',
        index=False,
        dtype= {
                "PartNum": sqlalchemy.types.VARCHAR(length=255),
             "EngPartNum": sqlalchemy.types.VARCHAR(length=255),
        }
    )


def get_inv():
    """Get Spare Inventory Data From iSeries AS400"""
    dbcnxn = pyodbc.connect(CONNAS400)
    cursor = dbcnxn.cursor()

    strsql = """SELECT PROD.FPSPRMAST1.SPH_PART,
                       STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC1),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC2),
                       STRIP(PROD.FPSPRMAST1.SPH_MFG),
                       STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
                       STRIP(PROD.FPSPRMAST2.SPD_CABINT),
                       STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
                       PROD.FPSPRMAST2.SPD_QOHCUR,
                       PROD.FPSPRMAST1.SPH_CURSTD,
                       STRIP(PROD.FPSPRMAST2.SPD_REODTE),
                       STRIP(PROD.FPSPRMAST2.SPD_USECC),
                       STRIP(PROD.FPSPRMAST2.SPD_PURCC),
                       STRIP(PROD.FPSPRMAST2.SPD_QREORD)
                FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
                WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))"""
    try:
        cursor.execute(strsql)
        result = cursor.fetchall()
    except Exception as e:
        msg = 'AS400 Inventory Query Failed: ' + str(e)
        result = []
        print(msg)
        print(strsql)
    else:
        msg = str(len(result)) + " AS400 Inventory Records Processed From Inventory Tables"
        print(msg)
    dbcnxn.close()
    return result

def find_new_obs_org(result_spares):
    """Find any newly obsoleted spare parts"""
    data_type_dict = {'StandardCost': float, 'OnHand': int, 'PartNum': str, 'ReOrderPt': int, 'ReOrderDate': int, 'Cabinet': str, 'Drawer': str}
    df_spares = pd.DataFrame.from_records(result_spares)
    df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand',
                         'StandardCost', 'ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
    df_spares = df_spares.dropna()
    df_spares = df_spares.astype(data_type_dict)
    df_spares = df_spares.convert_dtypes()
    df_obs_all = df_spares[df_spares.Cabinet.str.contains('OBS', case=False, na=False)]
    df_obs_current = pd.read_sql("SELECT PartNum FROM eng.tblObsSpares", engine)
    df_obs_new = df_obs_all[~df_obs_all['PartNum'].isin(df_obs_current['PartNum'])]
    return df_obs_new

def find_new_obs(result_spares):
    """Find any newly obsoleted spare parts"""
    data_type_dict = {'StandardCost': float, 'OnHand': int, 'PartNum': str, 'ReOrderPt': int,
                      'ReOrderDate': int, 'Cabinet': str, 'Drawer': str}
    # Ensure the correct structure of incoming data
    try:
        df_spares = pd.DataFrame.from_records(result_spares)
        expected_columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn',
                            'Cabinet', 'Drawer', 'OnHand', 'StandardCost', 'ReOrderDate',
                            'DeptUse', 'DeptPurch', 'ReOrderPt']
        if len(df_spares.columns) != len(expected_columns):
            raise ValueError("Mismatch in the number of columns in result_spares")
        df_spares.columns = expected_columns
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

    # Drop NaN values from critical columns
    df_spares = df_spares.dropna(subset=['PartNum', 'Cabinet', 'OnHand'])

    # Enforce data types safely
    for col, dtype in data_type_dict.items():
        if col in df_spares.columns:
            if dtype == int or dtype == float:
                df_spares[col] = pd.to_numeric(df_spares[col], errors='coerce')
            else:
                df_spares[col] = df_spares[col].astype(dtype)

    # Filter for obsolete parts
    df_obs_all = df_spares[df_spares.Cabinet.str.contains('OBS', case=False, na=False)]

    try:
        df_obs_current = pd.read_sql("SELECT PartNum FROM eng.tblObsSpares", engine)
    except Exception as e:
        print(f"Database query error: {e}")
        return pd.DataFrame()  # Return an empty DataFrame

    # Identify new obsolete parts
    df_obs_new = df_obs_all[~df_obs_all['PartNum'].isin(df_obs_current['PartNum'])]
    return df_obs_new


def send_email(to, subject, body, content_type='html', username='sgilmour@idealtridon.com'):
    # Send Email
    mail_server = "cas2013.ideal.us.com"

    if isinstance(to, list):
        # Join the list of email addresses into a single string
        to = ', '.join(to)


    try:
    # Create a MIME email
        message = MIMEMultipart()
        message['From'] = username
        message['To'] = to
        message['Subject'] = subject
        body = body + '<br><b>Sincerely,<br><br><br> The Engineering Overlords and Steve</b><br>'
        body = body + '<br><br><a href="https://www.idealtridon.com/idealtridongroup.html"> ' \
                        '<img src="https://sgilmo.com/email_logo.png" alt="Ideal Logo"></a>'
        # Attach the body content (HTML or plain text)
        message.attach(MIMEText(body, content_type))


        # Set up the SMTP connection
        with smtplib.SMTP(mail_server) as server:
            server.send_message(message)  # Send the email

        print(f"Email sent successfully to {to} with subject: {subject}")

    except Exception as e:
        print(f"Failed to send email: {e}")





In [48]:
# SQLAlchemy connection


df_obs_spares = find_new_obs(get_inv())
df_obs_nums = df_obs_spares[['PartNum', 'EngPartNum']]

# Renaming specific columns
df_obs_spares = df_obs_spares.rename(columns={'PartNum': 'Part Number', 'EngPartNum': 'Eng Part Number',
                                              'Desc1': 'Description 1', 'Desc2': 'Description 2'})

mail_list = ['sgilmour@idealtridon.com', 'bbrackman@idealtridon.com']

if df_obs_spares.size > 0:
    df_html_table = df_obs_spares.iloc[:, :4].to_html(index=False, border=1)
    send_email(mail_list, 'The Following Items Were Just Made Obsolete', df_html_table)
    add_obs(df_obs_nums)
    print(df_obs_nums)
else:
    print("No New Obsolete Parts")
# print(df_obs_spares.head())
# Write DataFrame to SQL Server table


# try:
#     df_obs_spares.to_sql(
#         name=table_name,
#         con=engine,
#         schema='eng',
#         if_exists='append',
#         index=False,
#         dtype= {
#                 "PartNum": sqlalchemy.types.VARCHAR(length=255),
#                 "EngPartNum": sqlalchemy.types.VARCHAR(length=255),
#         }
#     )
#     print(f"DataFrame successfully written to table '{table_name}' in the database '{database}'.")
# except sqlalchemy.exc.IntegrityError as e:
#     prob_desc = 'Problem Sending Data to SQL Server Table (Duplicate Primary Key)'
#     prob_detail = '<b>Problem Detail: </b><br>' + str(e)
#     print(prob_desc)
#     print(prob_detail)
# except Exception as e:
#     print(f"An error occurred: {e}")

22773 AS400 Inventory Records Processed From Inventory Tables
Email sent successfully to sgilmour@idealtridon.com, bbrackman@idealtridon.com with subject: The Following Items Were Just Made Obsolete
       PartNum EngPartNum
15987  1021016  091021016
16108  1021014  091021014
16374  1021013  091021013
16881  1021022  091021022
22170  1017050  091017050


In [ ]:
test_data = {
    'PartNum': ['123456', '65432'],
    'EngPartNum': ['abcdef', 'zxcvb']
}
sql_dtype= {
                "PartNum": sqlalchemy.types.VARCHAR(length=255),
                "EngPartNum": sqlalchemy.types.VARCHAR(length=255),
        }

test_df = pd.DataFrame(test_data)
print(test_df)

test_df.to_sql(name="tblObsSpares",con=engine,if_exists='replace',index=False,dtype=sql_dtype)